In [42]:
pip install snowflake-connector-python snowflake-snowpark-python python-dotenv pandas numpy matplotlib seaborn ydata-profiling faker notebook ipython ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [43]:
import snowflake.connector
import pandas as pd
from faker import Faker
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime, timedelta
import random



# Connect to Snowflake
ctx = snowflake.connector.connect(
    user="NGOMANE01",
    password="P.Ngomane@2025",
    account="KHWZLWE-TY26316",
    warehouse="COMPUTE_WH",
    database="TRICORE_TECH",
    schema="PUBLIC"
)

cs = ctx.cursor()

try:
    # Use your warehouse, db, schema
    cs.execute("USE WAREHOUSE COMPUTE_WH;")
    cs.execute("USE DATABASE TRICORE_TECH;")
    cs.execute("USE SCHEMA PUBLIC;")
    
 # Example: Fetch data from Loan_Application table
    cs.execute("SELECT * FROM HEALTH_SECTOR_INITIATIVES;")
    rows = cs.fetchall()
 
    # Convert to DataFrame (optional)
    columns = [desc[0] for desc in cs.description]
    df = pd.DataFrame(rows, columns=columns)
  
finally:
    cs.close()

In [44]:
df

,PATIENT_ID,NAME,PHONE,LANGUAGE,APPOINTMENT_DATE,CLINIC_NAME,LOCATION,CREATED_AT
0,P0001,Kyle Bernard,624396650,Zulu,2025-07-28,George Mukhari Hospital,Hazyview,2025-07-02 09:32:32
1,P0002,John Coleman,657371973,English,2025-08-15,Dora Nginza Hospital,Hazyview,2025-07-02 09:32:32
2,P0003,Mark Martin,645736400,Zulu,2025-08-16,Rahima Moosa Mother and Child Hospital,Qwebega,2025-07-02 09:32:32
3,P0004,Julie Rivera,699295597,IsiXhosa,2025-08-30,Dora Nginza Hospital,Qwebega,2025-07-02 09:32:32
4,P0005,Jeffrey Sanchez,737528016,IsiXhosa,2025-09-08,Inkosi Albert Luthuli Central Hospital,Hazyview,2025-07-02 09:32:32
...,...,...,...,...,...,...,...,...
545,P0546,Mary Tyler,652022713,English,2025-08-05,Inkosi Albert Luthuli Central Hospital,Hazyview,2025-07-02 09:32:32
546,P0547,Chelsea Jennings,799302226,English,2025-09-13,Mthatha General Hospital,Hazyview,2025-07-02 09:32:32
547,P0548,Jeffrey Lawson,765239540,IsiXhosa,2025-07-24,George Mukhari Hospital,Hazyview,2025-07-02 09:32:32
548,P0549,Carmen Thomas,615962586,Zulu,2025-07-07,Tygerberg Hospital,Hazyview,2025-07-02 09:32:32


In [45]:
df.columns

Index(['PATIENT_ID', 'NAME', 'PHONE', 'LANGUAGE', 'APPOINTMENT_DATE',
       'CLINIC_NAME', 'LOCATION', 'CREATED_AT'],
      dtype='object')

## Parsing columns

In [47]:

# Parse CREATED_AT column as datetime
df['CREATED_AT'] = pd.to_datetime(df['CREATED_AT'])

# Split into two new columns
df['CREATED_DATE'] = df['CREATED_AT'].dt.date
df['CREATED_TIME'] = df['CREATED_AT'].dt.time


# Drop original CREATED_AT if you want
df.drop(columns=['CREATED_AT'], inplace=True)



In [48]:
df

,PATIENT_ID,NAME,PHONE,LANGUAGE,APPOINTMENT_DATE,CLINIC_NAME,LOCATION,CREATED_DATE,CREATED_TIME
0,P0001,Kyle Bernard,624396650,Zulu,2025-07-28,George Mukhari Hospital,Hazyview,2025-07-02,09:32:32
1,P0002,John Coleman,657371973,English,2025-08-15,Dora Nginza Hospital,Hazyview,2025-07-02,09:32:32
2,P0003,Mark Martin,645736400,Zulu,2025-08-16,Rahima Moosa Mother and Child Hospital,Qwebega,2025-07-02,09:32:32
3,P0004,Julie Rivera,699295597,IsiXhosa,2025-08-30,Dora Nginza Hospital,Qwebega,2025-07-02,09:32:32
4,P0005,Jeffrey Sanchez,737528016,IsiXhosa,2025-09-08,Inkosi Albert Luthuli Central Hospital,Hazyview,2025-07-02,09:32:32
...,...,...,...,...,...,...,...,...,...
545,P0546,Mary Tyler,652022713,English,2025-08-05,Inkosi Albert Luthuli Central Hospital,Hazyview,2025-07-02,09:32:32
546,P0547,Chelsea Jennings,799302226,English,2025-09-13,Mthatha General Hospital,Hazyview,2025-07-02,09:32:32
547,P0548,Jeffrey Lawson,765239540,IsiXhosa,2025-07-24,George Mukhari Hospital,Hazyview,2025-07-02,09:32:32
548,P0549,Carmen Thomas,615962586,Zulu,2025-07-07,Tygerberg Hospital,Hazyview,2025-07-02,09:32:32


In [49]:
df.isnull().sum()

PATIENT_ID          0
NAME                0
PHONE               0
LANGUAGE            0
APPOINTMENT_DATE    0
CLINIC_NAME         0
LOCATION            0
CREATED_DATE        0
CREATED_TIME        0
dtype: int64

In [50]:
df.dtypes

PATIENT_ID          object
NAME                object
PHONE                int64
LANGUAGE            object
APPOINTMENT_DATE    object
CLINIC_NAME         object
LOCATION            object
CREATED_DATE        object
CREATED_TIME        object
dtype: object

In [51]:
# Convert PHONE to string
df['PHONE'] = df['PHONE'].astype(str)

# Overwrite PHONE column with +27 version
df['PHONE'] = df['PHONE'].apply(
    lambda x: '+27' + x[1:] if x.startswith('0') else '+27' + x
)


In [52]:
df

,PATIENT_ID,NAME,PHONE,LANGUAGE,APPOINTMENT_DATE,CLINIC_NAME,LOCATION,CREATED_DATE,CREATED_TIME
0,P0001,Kyle Bernard,+27624396650,Zulu,2025-07-28,George Mukhari Hospital,Hazyview,2025-07-02,09:32:32
1,P0002,John Coleman,+27657371973,English,2025-08-15,Dora Nginza Hospital,Hazyview,2025-07-02,09:32:32
2,P0003,Mark Martin,+27645736400,Zulu,2025-08-16,Rahima Moosa Mother and Child Hospital,Qwebega,2025-07-02,09:32:32
3,P0004,Julie Rivera,+27699295597,IsiXhosa,2025-08-30,Dora Nginza Hospital,Qwebega,2025-07-02,09:32:32
4,P0005,Jeffrey Sanchez,+27737528016,IsiXhosa,2025-09-08,Inkosi Albert Luthuli Central Hospital,Hazyview,2025-07-02,09:32:32
...,...,...,...,...,...,...,...,...,...
545,P0546,Mary Tyler,+27652022713,English,2025-08-05,Inkosi Albert Luthuli Central Hospital,Hazyview,2025-07-02,09:32:32
546,P0547,Chelsea Jennings,+27799302226,English,2025-09-13,Mthatha General Hospital,Hazyview,2025-07-02,09:32:32
547,P0548,Jeffrey Lawson,+27765239540,IsiXhosa,2025-07-24,George Mukhari Hospital,Hazyview,2025-07-02,09:32:32
548,P0549,Carmen Thomas,+27615962586,Zulu,2025-07-07,Tygerberg Hospital,Hazyview,2025-07-02,09:32:32


In [53]:
import random



# Common elderly diseases
elderly_diseases = [
    "Hypertension", "Diabetes", "Arthritis", "Asthma", "Alzheimer's",
    "Chronic Kidney Disease", "Heart Disease",
    "Cataracts", "Hearing Loss", "Glaucoma", "Parkinson's",
    "COPD", "Depression", "Prostate Enlargement", "Stroke Recovery", "High Cholesterol"
]
# Add AGE column with random ages 60-90
ages = [random.randint(60, 90) for _ in range(len(df))]
df['AGE'] = ages

# Move AGE column right after NAME
cols = df.columns.tolist()
cols.insert(cols.index('NAME') + 1, cols.pop(cols.index('AGE')))
df = df[cols]
# Generate diseases column
df['DISEASES'] = [", ".join(random.sample(elderly_diseases, random.randint(1, 3))) for _ in range(len(df))]

# Reorder columns to place DISEASES right after NAME
cols = df.columns.tolist()
cols.insert(cols.index('AGE') + 1, cols.pop(cols.index('DISEASES')))
df = df[cols]

C:\Users\CAPACITI\AppData\Local\Temp\ipykernel_7420\1310253269.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DISEASES'] = [", ".join(random.sample(elderly_diseases, random.randint(1, 3))) for _ in range(len(df))]


In [54]:
# Helper to generate random visit time between 08:00 and 16:00 on 15-min intervals
def random_visit_time():
    hour = random.randint(8, 16)
    minute = random.choice([0, 15, 30, 45])
    return f"{hour:02d}:{minute:02d}"

# Generate LAST_VISIT datetime (date + time)
last_visits = []
last_visit_times = []
for _ in range(len(df)):
    date = datetime.today() - timedelta(days=random.randint(30, 365))
    time_str = random_visit_time()
    time_obj = datetime.strptime(time_str, "%H:%M").time()
    last_visits.append(date.date())
    last_visit_times.append(time_obj.strftime("%H:%M"))

df['LAST_VISIT'] = last_visits
df['LAST_VISIT_TIME'] = last_visit_times

# Generate NEXT_VISIT datetime (30 to 90 days after LAST_VISIT + random time)
next_visits = []
next_visit_times = []
for lv in last_visits:
    next_date = lv + timedelta(days=random.randint(30, 90))
    time_str = random_visit_time()
    time_obj = datetime.strptime(time_str, "%H:%M").time()
    next_visits.append(next_date)
    next_visit_times.append(time_obj.strftime("%H:%M"))

df['NEXT_VISIT'] = next_visits
df['NEXT_VISIT_TIME'] = next_visit_times


# Reorder columns to keep visits grouped nicely
cols = df.columns.tolist()
for col in ['LAST_VISIT', 'LAST_VISIT_TIME', 'NEXT_VISIT', 'NEXT_VISIT_TIME']:
    cols.insert(cols.index('APPOINTMENT_DATE') + 1, cols.pop(cols.index(col)))
clean = df[cols]

In [55]:



# Assuming your DataFrame is 'clean'
clean = clean.copy()  # Make a proper independent copy first
clean.loc[:, 'CLINIC_VISITS'] = [random.randint(1, 12) for _ in range(len(clean))]

cols = clean.columns.tolist()
cols.insert(cols.index('APPOINTMENT_DATE') + 1, cols.pop(cols.index('CLINIC_VISITS')))
clean = clean[cols]

In [56]:
clean.columns

Index(['PATIENT_ID', 'NAME', 'AGE', 'DISEASES', 'PHONE', 'LANGUAGE',
       'APPOINTMENT_DATE', 'CLINIC_VISITS', 'NEXT_VISIT_TIME', 'NEXT_VISIT',
       'LAST_VISIT_TIME', 'LAST_VISIT', 'CLINIC_NAME', 'LOCATION',
       'CREATED_DATE', 'CREATED_TIME'],
      dtype='object')

In [57]:
# Overwrite the table with the cleaned DataFrame
success, nchunks, nrows, _ = write_pandas(
    ctx,
    clean,
    'Clean_Health_data',
    overwrite=True,        # Overwrite the existing table
    auto_create_table=True # Create the table if it doesn't exist
)
print(f"Success: {success}, Chunks: {nchunks}, Rows: {nrows}")
ctx.close()

Success: True, Chunks: 1, Rows: 550


In [58]:
pip install fastapi

Note: you may need to restart the kernel to use updated packages.


In [59]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import snowflake.connector
import pandas as pd

# 🔧 Initialize FastAPI app
app = FastAPI(
    title="Snowflake API",
    description="Exposes Snowflake data to your frontend",
    version="1.0"
)

# 🌐 Enable CORS (so your website can access the API)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In prod: ["https://yourdomain.com"]
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 🔌 Connect to Snowflake & fetch data
def fetch_health_data():
    conn = snowflake.connector.connect(
        user="NGOMANE01",
        password="P.Ngomane@2025",
        account="KHWZLWE-TY26316",
        warehouse="COMPUTE_WH",
        database="TRICORE_TECH",
        schema="PUBLIC"
    )

    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM HEALTH_SECTOR_INITIATIVES;")
        columns = [desc[0] for desc in cursor.description]
        rows = cursor.fetchall()
        df = pd.DataFrame(rows, columns=columns)
        return df
    finally:
        cursor.close()
        conn.close()

# ✅ API route
@app.get("/health-data")
def get_health_sector_data():
    df = fetch_health_data()
    return df.to_dict(orient="records")
    
@app.get("/")
def root():
    return {"message": "Snowflake API is live. Use /health-data to get data."}


In [60]:
pip install fastapi uvicorn snowflake-connector-python pandas


Note: you may need to restart the kernel to use updated packages.


In [61]:
import requests

response = requests.get("https://snowflake-api-1.onrender.com/health-data")
print(response.status_code)
print(response.json())


200
[{'PATIENT_ID': 'P0001', 'NAME': 'Kyle Bernard', 'PHONE': 624396650, 'LANGUAGE': 'Zulu', 'APPOINTMENT_DATE': '2025-07-28', 'CLINIC_NAME': 'George Mukhari Hospital', 'LOCATION': 'Hazyview', 'CREATED_AT': '2025-07-02T09:32:32'}, {'PATIENT_ID': 'P0002', 'NAME': 'John Coleman', 'PHONE': 657371973, 'LANGUAGE': 'English', 'APPOINTMENT_DATE': '2025-08-15', 'CLINIC_NAME': 'Dora Nginza Hospital', 'LOCATION': 'Hazyview', 'CREATED_AT': '2025-07-02T09:32:32'}, {'PATIENT_ID': 'P0003', 'NAME': 'Mark Martin', 'PHONE': 645736400, 'LANGUAGE': 'Zulu', 'APPOINTMENT_DATE': '2025-08-16', 'CLINIC_NAME': 'Rahima Moosa Mother and Child Hospital', 'LOCATION': 'Qwebega', 'CREATED_AT': '2025-07-02T09:32:32'}, {'PATIENT_ID': 'P0004', 'NAME': 'Julie Rivera', 'PHONE': 699295597, 'LANGUAGE': 'IsiXhosa', 'APPOINTMENT_DATE': '2025-08-30', 'CLINIC_NAME': 'Dora Nginza Hospital', 'LOCATION': 'Qwebega', 'CREATED_AT': '2025-07-02T09:32:32'}, {'PATIENT_ID': 'P0005', 'NAME': 'Jeffrey Sanchez', 'PHONE': 737528016, 'LANGUA